Trouble shoot pyecharts.

Related issue:

- [#104](https://github.com/hupili/python-for-data-and-media-communication-gitbook/issues/104)

In [18]:
import pyecharts

In [19]:
import datetime

In [20]:
pyecharts.__version__

'0.5.11'

In [23]:
datetime.datetime.now()

datetime.datetime(2018, 11, 24, 0, 24, 4, 609665)

## The failed trial

In [3]:
import pandas as pd
import csv
from dateutil.parser import parse
df = pd.read_csv('Billboard Top1 1958-2018.csv')
df['datetime']=df['date'].apply(parse)
describe_song=df.set_index('datetime').resample('1y')['song'].describe()
describe_song.head()

hhh=pd.DataFrame(pd.to_numeric(describe_song['unique']))
years = list(hhh.reset_index()['datetime'].apply(lambda x: x.year).values)
years
values = list(hhh.unique.values)
values
from pyecharts import Line

line = Line("Unique change trend")
line.add("Unique Songs", years, values, mark_line=["average"], mark_point=['max','min'])
line

In [4]:
af=df.set_index('datetime').resample('10y')['song'].value_counts()
af.to_csv('topsongs_by_decade.csv')
songs = pd.read_csv('topsongs_by_decade.csv', header=None,names = ["datetime", "titles", "counts"])

## Trouble shoot

You always want to start with simple cases. If the data table can not be plotted, let's try to plot some artificial data first. We manually fill in some data for index and values.

In [5]:
from pyecharts import Line

line = Line("Unique change trend")
line.add("Unique Songs", [1990, 2000, 2010], [10, 30, 20], mark_line=["average"], mark_point=['max','min'])
line

The above does not work. So we way of using pyecharts may be wrong. Let see if other chart type works if not `Line`.

In [8]:
from pyecharts import Bar

bar = Bar("Unique change trend")
bar.add("Unique Songs", [1990, 2000, 2010], [10, 30, 20], mark_line=["average"], mark_point=['max','min'])
bar

In [9]:
from pyecharts import Bar

bar = Bar("Unique change trend")
bar.add("Unique Songs", years, values, mark_line=["average"], mark_point=['max','min'])
bar

Both artificial data and real data work on bar chart.

Then, let's get back to the line chart. See if the simplified version works.

In [12]:
from pyecharts import Line

line = Line("Unique change trend")
line.add("Unique Songs", [1990, 2000, 2010], [10, 30, 20]) #, mark_line=["average"], mark_point=['max','min'])
line.render('test.html')

## Solution 1 - adapt to pyecharts

In [14]:
from pyecharts import Line

line = Line("Unique change trend")
line.add("Unique Songs", ['1990', '2000', '2010'], [10, 30, 20]) #, mark_line=["average"], mark_point=['max','min'])
line

In [15]:
from pyecharts import Line

line = Line("Unique change trend")
line.add("Unique Songs", [str(y) for y in years], values) #, mark_line=["average"], mark_point=['max','min'])
line

In [17]:
#years

## Solution 2 - hack pyecharts

In [25]:
#pyecharts

Let's put something that does not work first:

In [26]:
from pyecharts import Line

line = Line("Unique change trend")
line.add("Unique Songs", [1990, 2000, 2010], [10, 30, 20]) #, mark_line=["average"], mark_point=['max','min'])
line

In [28]:
line._option

{'animation': True,
 'backgroundColor': None,
 'color': ['#c23531',
  '#2f4554',
  '#61a0a8',
  '#d48265',
  '#749f83',
  '#ca8622',
  '#bda29a',
  '#6e7074',
  '#546570',
  '#c4ccd3',
  '#f05b72',
  '#ef5b9c',
  '#f47920',
  '#905a3d',
  '#fab27b',
  '#2a5caa',
  '#444693',
  '#726930',
  '#b2d235',
  '#6d8346',
  '#ac6767',
  '#1d953f',
  '#6950a1',
  '#918597',
  '#f6f5ec'],
 'legend': [{'data': ['Unique Songs'],
   'left': 'center',
   'orient': 'horizontal',
   'selectedMode': 'multiple',
   'show': True,
   'textStyle': {'color': None, 'fontSize': 12},
   'top': 'top'}],
 'series': [{'areaStyle': {'color': None, 'opacity': 0},
   'data': [[1990, 10], [2000, 30], [2010, 20]],
   'label': {'emphasis': <pyecharts.echarts.label.EmphasisLabel at 0x1104bd9e8>,
    'normal': <pyecharts.echarts.label.NormalLabel at 0x1104bda90>},
   'lineStyle': {'normal': <pyecharts.echarts.line.Line at 0x1104bd128>},
   'markLine': {'data': []},
   'markPoint': {'data': []},
   'name': 'Unique Songs',


In [31]:
xAxis = line._option['xAxis'][0]

In [34]:
xAxis.__dict__

{'_config': {'axisLabel': <pyecharts.echarts.axis.XAxisLabel at 0x1104bdac8>,
  'axisLine': {'lineStyle': {'color': None, 'width': 1}},
  'axisTick': {'alignWithLabel': False},
  'boundaryGap': True,
  'data': [1990, 2000, 2010],
  'inverse': False,
  'max': None,
  'min': None,
  'name': '',
  'nameGap': 25,
  'nameLocation': 'middle',
  'nameTextStyle': {'fontSize': 14},
  'position': None,
  'show': True,
  'splitLine': {'show': False},
  'type': 'category'}}

In [35]:
xAxis['type'] = 'value'

In [36]:
line

Nothing in the above because the rendered HTML is already associated with a previous Cell. That is an echarts and Jupyter notebook interworking issue.

However, we know this method works. So let's put up the hack here and make a correct use of line chart.

In [37]:
from pyecharts import Line

line = Line("Unique change trend")
line.add("Unique Songs", [1990, 2000, 2010], [10, 30, 20]) #, mark_line=["average"], mark_point=['max','min'])
xAxis = line._option['xAxis'][0]
xAxis['type'] = 'value'
xAxis['min'] = 1980
xAxis['max'] = 2020
line

Bingo! Then we try the real data.

In [38]:
from pyecharts import Line

line = Line("Unique change trend")
line.add("Unique Songs", years, values) #, mark_line=["average"], mark_point=['max','min'])
xAxis = line._option['xAxis'][0]
xAxis['type'] = 'value'
xAxis['min'] = min(years)
xAxis['max'] = max(years)
line